In [ ]:
# !pip install handprint

# handprint -a google creds/handprint_google_credentials.json

# fix protobuf
# pip uninstall protobuf
# pip install protobuf==3.20.*

EXtract images from pdf

In [ ]:
import fitz
import math


pdffile = "input.pdf"
doc = fitz.open(pdffile)
for i in range(doc.page_count):
    page = doc.load_page(i)  # number of page
    pix = page.get_pixmap()
    output = f"image_output/outfile_i{i}_student{math.floor(i/2)}_{i % 2}.png"
    pix.save(output)

doc.close()

RED PEN REMOVER

In [ ]:
from PIL import Image, ImageEnhance                                                                
import sys
import os

In [ ]:
image_output_dir = "image_output/"
output_dir = "color_correction_output/"
for name in os.listdir(image_output_dir):
    img = Image.open(image_output_dir + name)
    img = img.convert("RGBA")

    pixdata = img.load()

    # Clean the background noise, if color != white, then set to black.

    # REMOVE RED PEN
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            r, g, b, a = pixdata[x, y]
            
            # REMOVE RED PEN
            if (r - g > 20 and
                r - b > 20 and
                r > 200) :
                
                radius = 2
                
                for i in range(2*radius):
                    for j in range(2*radius):
                        pixdata[x + i - radius, y + j - radius] = (255, 255, 255)

    # img.convert("L")

    # SHARPEN
    enhancer = ImageEnhance.Sharpness(img)
    img = enhancer.enhance(10)

    if (not os.path.exists(output_dir)):
        os.makedirs(output_dir)

    img.save(output_dir + name.replace('outfile_', 'colcor_'))

sections

In [1]:
from PIL import Image, ImageDraw
import os
import shutil
import json 
from pprint import pprint


current_path = os.getcwd() 

input_path = current_path + r"/color_correction_output"
temp_path = current_path + r"/temp_pre_handprint"
output_dir = current_path + r"/handprint_output"
section_output = current_path + r"/section_output"

try:
    os.makedirs(temp_path)
except:
    pass

try:
    os.makedirs(output_dir)
except:
    pass

def move_file(old_path, new_path):
    os.rename(old_path, new_path)
    # os.replace(old_path, new_path)
    shutil.move(old_path, new_path)

for name in os.listdir(input_path):
    
    image = Image.open(input_path+"/"+name)
    # only take the sidebar
    image = image.crop((0,0,105,image.height))
    print(f'Sidebar ... {temp_path+"/"+name}')

    image.save(temp_path+"/"+name.replace('colcor_', 'sidebar_'))
    print(f'Sidebar ... done')

for name in os.listdir(temp_path):
    output_path = output_dir +"/" + name
    if (not os.path.exists(output_path)):
        print(f'Starting ... {output_path}')
        # command = f"handprint /o {output_dir} /e /s google {temp_path}/{name} "
        command = f"handprint /o {output_dir} /e /s microsoft {temp_path}/{name} "
        print(command)
        !{command}
        print(f'Starting ... done')
    else:
        print(f"path already exists: {output_path} ")
    
for name in os.listdir(input_path):
    if name.startswith('textoutput'):
        move_file(input_path+"/"+name, output_dir+"/"+name)

Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i0_student0_0.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i100_student50_0.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i101_student50_1.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i102_student51_0.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i103_student51_1.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i104_student52_0.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i105_student52_1.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i106_student53_0.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i107_student53_1.png
Sidebar ... done
Sidebar ... d:\GitHub\PWS-inscannen/temp_pre_handprint/colcor_i108_student54_0.png
Sidebar ... done
Sid

In [2]:
from PIL import Image, ImageDraw
import re
import os 

current_path = os.getcwd() 

input_path = current_path + r"/color_correction_output"

output_dir = current_path + r"/handprint_output"

try:
    os.makedirs(output_dir)
except:
    pass
output_items = os.listdir(output_dir)



def filenameify(s):
    replace_dict = {
        "①": "1",
        "②": "2",
        "③": "3",
        "④": "4",
        "⑤": "5",
        "⑥": "6",
        "⑦": "7",
        "⑧": "8",
        "⑨": "9",
        "⑩": "10",
        "⑪": "11",
        "⑫": "12",
        "⑬": "13",
        "⑭": "14",
        "⑮": "15",
        "⑯": "16",
        "⑰": "17",
        "⑱": "18",
        "⑲": "19",
        "⑳": "20",
        ".": ""
    }
    
    # https://www.geeksforgeeks.org/python-replace-words-from-dictionary/
    replaced_s = " ".join(replace_dict.get(ele, ele) for ele in s.split())
    
    filename = re.sub(r'[^a-zA-Z0-9._-]', '', replaced_s)
    
    if (len(filename) == 0):
        return "NOT_VALID_QUESTION_ID"
    
    return filename

def clamp(n, min, max): 
    if n < min: 
        return min
    elif n > max: 
        return max
    else: 
        return n 

for file_name in os.listdir(input_path):
    clean_file_name = file_name.split(".")
    clean_file_name.pop()
    name = ".".join(clean_file_name).replace("colcor_", "sidebar_")
    
    output_json_file_name = name + ".handprint-google.json"
    output_image_file_name = name + ".handprint-google.png"
    output_name = name.replace('sidebar_', '')
    if (output_json_file_name not in output_items):
        continue
    
    # output json exists
    with open(output_dir+"/"+output_json_file_name, "r") as f:
        # file_text = f.read()
        text_data = json.load(f)
    
    # print(json.dumps(text_data["text_annotations"][0], sort_keys=True, indent=4))
    horizontal_breaks = []
    for data in text_data["text_annotations"]:
        if (data["bounding_poly"]["vertices"][1]["x"] < 115):
            # if (data["description"].isnumeric()):
            horizontal_breaks.append({"height": data["bounding_poly"]["vertices"][0]["y"], "description": data["description"]})
    # print(horizontal_breaks)
    
    
    image = Image.open(input_path+"/"+file_name)
    # to bottom of image
    horizontal_breaks.append({"height": image.height, "description": "None"})

    output_image = ImageDraw.Draw(image)

    
    for h_break in horizontal_breaks:
        y = h_break["height"]
        # output_image.line([(0, y-15), (image.width, y-15)], fill="red", width=3,)

    main_directory = section_output+"/"+output_name
    try:
        os.makedirs(main_directory)
    except:
        pass
    # image.show()
    image.save(main_directory+"/"+output_name+'.png')
    
    for index, h_break in enumerate(horizontal_breaks):
        if (index >= len(horizontal_breaks) - 1 ):
            continue
        y = clamp(h_break["height"] - 15, 0, image.height - 1)
        next_y = clamp(horizontal_breaks[index+1]["height"], 0, image.height - 1)
        
        # min height and fix
        if (next_y < y or next_y - y < 30):
            continue
        # kantlijn, boven, einde van pagina, beneden grens
        crop = (0, y, image.width, next_y)
        
        section_image = image.copy()
        cropped = section_image.crop(crop)
        section_directory = main_directory+"/sections"
        try:
            os.makedirs(section_directory)
        except:
            pass
        # section_image.show()
        section_file_name = filenameify(h_break["description"])+".png"
        try:
            cropped.save(section_directory+"/"+section_file_name)
            print(f'Saving {section_directory}/{section_file_name}')
        except:
            print("Could not save: ", section_file_name)
    
    print("Finished image: "+output_name)


OPEN AI tekstherkenning

In [3]:
from openai import OpenAI
import base64
import tiktoken
import json
from PIL import Image
import math
from pydantic import BaseModel
from io import BytesIO
import os
import shutil
import time


current_path = os.getcwd() 

def num_tokens_from_messages(messages, model="gpt-3.5-turbo-0613"):
    """Returns the number of tokens used by a list of messages."""
    #   https://github.com/openai/tiktoken?tab=readme-ov-file#-tiktoken
    try:
        encoding = tiktoken.encoding_for_model(model)
    except KeyError:
        encoding = tiktoken.get_encoding("cl100k_base")
        
    num_tokens = 0
    for message in messages:
        num_tokens += 4  # every message follows <im_start>{role/name}\n{content}<im_end>\n
        for data_value in message["content"]:
            if (data_value["type"] == "text"):
                data = data_value["text"]
            
            if (data_value["type"] == "image_url"):
                data = data_value["image_url"]["url"]    
            if (data):
                num_tokens += len(encoding.encode(data))
            # if data_value["type"] == "name":  # if there's a name, the role is omitted
            #     num_tokens += -1  # role is always required and always 1 token
    num_tokens += 2  # every reply is primed with <im_start>assistant
    return num_tokens

# https://platform.openai.com/docs/guides/vision 
def encode_image(image_path):
    pillow_image = Image.open(image_path)
    
    percent = 1

    new_width = int(pillow_image.width * percent)
    new_height = int(float(pillow_image.size[1]) * percent)
    
    resized_image = pillow_image.resize((new_width, new_height))
    buffered = BytesIO()
    resized_image.save(buffered, format="PNG")
    return base64.b64encode(buffered.getvalue()).decode('utf-8')

def copy_image(image_path, target_path):
    shutil.copy(image_path, target_path)


# schema classes
class SpellingCorrection(BaseModel):
    original: str
    changes: str

# answer class schema
class QuestionAnswer(BaseModel):
    # let openai reasses the question number (they are better than google )
    question_number: int 
    # get the unchanged raw tekst
    raw_text: str
    # get the spel corrected tekst that should be graded
    correctly_spelled_text: str
    # get the spelling changes the model made
    spelling_corrections: list[SpellingCorrection]

def scan_section(student, image_name):

    input_image_path = f"section_output/{student}/sections/{image_name}.png"

    base64_image = encode_image(input_image_path)

    messages = [
        {
            "role": "system",
            "content": [
                {
                    "type": "text",
                    "text": """Je krijgt een foto van een Nederlandse toetsantwoord. 
                                Je moet deze omzetten in tekst. 
                                Deze toets moet nog worden nagekeken je. 
                                Verander niets aan de inhoud. 
                                Bedenk geen nieuwe woorden of woordonderdelen. 
                                Verander alleen kleine spelfoutjes.
                                houd in het antwoord ook rekening met meerdere regels en geeft die aan met een '\\n'
                                """
                },
                {
                    "type": "text",
                    "text": "Geef antwoord in JSON zoals in een aangegeven schema staat"    
                },
            ]
        },
        {
            "role": "user",
            "content": [
                {
                    "type": "text",
                    "text": "extraheer de tekst"    
                },
                {
                    "type": "image_url",
                    "image_url": {
                        "url": f"data:image/png;base64,{base64_image}"
                    }
                }
            ]
        }
    ]
    

    with open("creds/openaikey.json", "r") as f:
        openai_key = json.load(f)["key"]
        

    openai_client = OpenAI(api_key=openai_key)

    gpt_model = "gpt-4o-mini"

    num_tokens = num_tokens_from_messages(messages= messages,model="gpt-4o")

    # if (input(f"execute {num_tokens}? (y/n)") != "y" ):
        # exit()

    start_time = time.time()

    print("GPT request... ")
    try:
        response = openai_client.beta.chat.completions.parse(
            model=gpt_model,
            temperature=0.05,
            messages=messages,
            response_format=QuestionAnswer,
            # max_tokens=30_000, #test image was 15k tokens
            timeout=14
        )
    except:
        print("GPT request... ERROR")
        return False
    print("GPT request... Done")

    end_time = time.time()

    # from pprint import pprint
    # pprint(response)
    json_output_path = current_path + r"/json_output"


    try:
        os.makedirs(json_output_path)
    except:
        pass

    student_json_output_path = json_output_path + "/"+ student + "/sections" 
    try:
        os.makedirs(student_json_output_path)
    except:
        pass

    # TODO: choose better name
    #  currently using the original image file name
    image_json_output_path = student_json_output_path + "/" + image_name + '.png'
    try:
        os.makedirs(image_json_output_path)
    except:
        pass

    # copy image here
    copy_image(input_image_path, image_json_output_path)
        

    json_output_path = image_json_output_path + "/" + gpt_model + "_result.json"

    dict_response = response.to_dict()
    choice_response = dict_response["choices"][0]
    result_data = choice_response["message"]["parsed"]
    request_data = dict_response["usage"]

    output_json = {
        "google_quiestion_number": image_name,

        "question_number": result_data["question_number"],
        "raw_text": result_data["raw_text"],
        "correctly_spelled_text": result_data["correctly_spelled_text"],
        "spelling_corrections": result_data["spelling_corrections"],
        
        "tokens_used": request_data["total_tokens"],

        "model_used": gpt_model,
        "model_version": dict_response["model"],
        
        "timestamp": int(end_time),
        "delta_time_s": end_time - start_time,
    }



    json_object = json.dumps(output_json, indent=4)
    with open(json_output_path, "w") as outfile:
        outfile.write(json_object)
        
    print("Written to file "+json_output_path)

student = "i1_student0_1"
image_name = "10-"

# scan_section(student, image_name)


In [4]:
import os
from PIL import Image
import sys
import psutil

def ask_image(path) -> bool:
    img = Image.open(path)
    img.show()
    result = input("Send to gpt (y/n) 'quit' = quit: ")
    return result
    
i = 0    
if os.path.exists("section_output/"):
    for student in os.listdir("section_output/"):
        if (os.path.exists(f"section_output/{student}/sections")
            # dont to existing ones
            and not os.path.exists(f"json_output/{student}/sections/")):
            

            for section in os.listdir(f"section_output/{student}/sections/"):
                
                # scan = ask_image(f"section_output/{student}/sections/{section}")
                # control over usage of api
                # if scan == "y":
                clean_section_name = section.replace('.png', '' )
                
                i+=1
                print(f"Starting {i} - {student} - {section} ...")   
                scan_section(student, clean_section_name)
                print(f"Starting {i} - {student} - {section} ... Done")   

                # if (scan == "quit"):
                    # raise SystemExit(0)

Starting 1 - i0_student0_0 - -2.png ...
GPT request... 
GPT request... Done
Written to file d:\GitHub\PWS-inscannen/json_output/i0_student0_0/sections/-2.png/gpt-4o-mini_result.json
Starting 1 - i0_student0_0 - -2.png ... Done
Starting 2 - i0_student0_0 - 3.png ...
GPT request... 
GPT request... Done
Written to file d:\GitHub\PWS-inscannen/json_output/i0_student0_0/sections/3.png/gpt-4o-mini_result.json
Starting 2 - i0_student0_0 - 3.png ... Done
Starting 3 - i0_student0_0 - 33.png ...
GPT request... 
GPT request... Done
Written to file d:\GitHub\PWS-inscannen/json_output/i0_student0_0/sections/33.png/gpt-4o-mini_result.json
Starting 3 - i0_student0_0 - 33.png ... Done
Starting 4 - i0_student0_0 - 42.png ...
GPT request... 
GPT request... Done
Written to file d:\GitHub\PWS-inscannen/json_output/i0_student0_0/sections/42.png/gpt-4o-mini_result.json
Starting 4 - i0_student0_0 - 42.png ... Done
Starting 5 - i0_student0_0 - NOT_VALID_QUESTION_ID.png ...
GPT request... 
GPT request... Done


In [12]:
from PIL import  Image  



total_requests = 0
if os.path.exists("json_output/"):
    for student in os.listdir("json_output/"):
        if (os.path.exists(f"json_output/{student}/sections")):
            

            for section in os.listdir(f"json_output/{student}/sections/"):
                total_requests += 1
                
print(total_requests)


26


In [11]:
from PIL import  Image  



total_requests = 0
if os.path.exists("section_output/"):
    for student in os.listdir("section_output/"):
        if (os.path.exists(f"section_output/{student}/sections")
):
            

            for section in os.listdir(f"section_output/{student}/sections/"):
                total_requests += 1
                
print(total_requests)


567
